In [1]:
import numpy as np
import pandas as pd

In [2]:
# coding:utf-8
#from __future__ import print_function  # 必须放到开始
import numpy as np
import pandas as pd
import time

# 预设值
N_STATES = 6  # 状态数量
ACTIONS = ['left', 'right']  # 行为，两种。
EPSILON = 0.9  # epsilon greedy，贪婪度
ALPHA = 0.1  # 学习率
GAMMA = 0.9  # 奖励递减值，也就是不重视奖励的程度
MAX_EPISODES = 10  # 最大回合数
FRESH_TIME = 0.1  # 每一帧画面停留时间

In [3]:
# 初始化q_table
def build_q_table():
    q_table = pd.DataFrame(np.zeros((N_STATES, len(ACTIONS))), columns=ACTIONS)
    return q_table

In [14]:
s =q_table.iloc[2, :] 
s

left     0.0
right    0.0
Name: 2, dtype: float64

In [15]:
np.random.uniform()

0.09233899961749337

In [16]:
# 选择动作，action
# 利用epsilon greedy选择a
def choose_action(state, q_table):
    state_actions = q_table.iloc[state, :]
    if np.random.uniform() > EPSILON or is_all_zero(state_actions):#在某一概率或者在当前状态的移动没有转移概率的时候
        action_name = np.random.choice(ACTIONS)                    #随机选择action
    else:
        action_name = state_actions.idxmax() #按照某一概率使用贪婪算法选择较大概率的动作作为action执行
    return action_name  #action_name为left or right

In [17]:
q_table = build_q_table()
q_table
choose_action(0, q_table)
q_table

,left,right
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0


In [8]:
# 判断全零行
def is_all_zero(series):
    for s in series:
        if s != 0:
            return False
    return True

In [18]:
is_all_zero(s)

True

In [10]:
# 环境反馈
# 执行走步操作，从(S, A)到S_状态，并获得奖励R
# 得到S_、R
#定义获取reword的反馈
def get_env_feedback(state, action):
    if action == 'right':
        if state == N_STATES - 2:
            next_state = 'terminal'
            R = 1
        else:
            next_state = state + 1
            R = 0
    else:
        if state == 0:  # 最左侧的state没有办法在进行移动，如果在state等于0的状态下仍然选择左移，则默认不改变state；todo why?将if state 改成 if state == 0 就没有bug了？而之前next_state会变成-1
            next_state = state
        else:
            next_state = state - 1
        R = 0
    # print('\nS: {}-{}-S\': {}'.format(state, action, next_state))
    return next_state, R

In [11]:

# 环境更新
# 每走完一步，要更新一帧图
def update_env(state, episode, step_counter):
    env_list = ['-'] * (N_STATES - 1) + ['T']
    if state == 'terminal':
        interaction = 'episode: %s; total_steps = %s' % (episode + 1, step_counter)  # fixme +1？？？
        print('\r{}'.format(interaction),end ='')  # \r把之前打在屏幕上的信息给删掉了
        time.sleep(2)
        print('\r                           ', end='')  # 清屏
    else:
        env_list[state] = 'o'
#         print(env_list)
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction),
              end='')  # end=''是Python3的内容，必须在文件导入的部分第一句位置写from __future__ import print_function
        # \r是回车，回到一行的开始
        time.sleep(FRESH_TIME)

In [12]:
state, episode, step_counter = 2,0,0
update_env(state, episode, step_counter)

--o--T

In [13]:
# q_learning
def q_learing():
    q_table = build_q_table()#初始化一个状态表
    for episode in range(MAX_EPISODES):  # TODO 如果是判断Q阵收敛，可以判断reward不再发生变化
        step_counter = 0  # 走的步数
        state = 0
        is_terminated = False  # 一局游戏结束的标志
        update_env(state, episode, step_counter)
        #print(is_terminated)
        while not is_terminated:
            action = choose_action(state, q_table)
            # print('\nS值: {}\n'.format(state))
            # print(action)
            next_state, R = get_env_feedback(state, action)#与环境的交互
            q_predict = q_table.loc[state, action]
            #print(q_predict)
            if next_state != 'terminal':
                q_target = R + GAMMA * q_table.iloc[next_state, :].max() #没有到达终点，q = 奖励 + 学习率×下一状态的r最大值（下一状态的结果只与前一时刻状态有关）
                #print("when the next_state is not terminal,the q_target is".format(q_target))
            else:
                q_target = R  # 到终点直接拿糖
                #print("when the next_state is terminal,the q_target is".format(q_target))
                is_terminated = True
            #训练过程中，不断更新q_table表
            q_table.loc[state, action] = q_predict + ALPHA * (q_target - q_predict) #当前状态的q值更新 = q的预测值+学习率×目标值与预测值的差值
            # 如果不区分两种状态，是否可以如下写：
            # q_table.loc[state, action] = (1 - ALPHA) * q_table.loc[state, action] + ALPHA * (
            #         R + GAMMA * q_table.loc[next_state, :].max())
            # print('\nS\'值: {}\n'.format(state))
            state = next_state
            step_counter += 1
            update_env(state, episode, step_counter)
    return q_table

####  整体流程梳理：
1. 初始化一个状态表；
2. 在episode内进行状态转换（在每一次迭代过程中reward不在发生变化时，不再训练）；（重复步骤3，4,5）
    3. 更新环境展示；
    4. 当前状态不处于终止状态时,
        选择动作->根据当前状态与动作从q根据反馈值，选择下一时刻状态与reward；
        5. 从q表中根据当前状态与动作获取q值；
        6. 根据当前状态获取q_tar的值。
        7. 更新q表，（根据下一时刻的状态来更新当前的q值）
        8. 更新状态和外部环境展示

In [18]:
if __name__ == '__main__':
    q_table = q_learing()
    print('\nQ_Table:\n{}'.format(q_table))

                           6
Q_Table:
       left     right
0  0.000103  0.001875
1  0.000000  0.012989
2  0.000083  0.063479
3  0.002674  0.237511
4  0.000000  0.651322
5  0.000000  0.000000
